# Лабораторная работа №2
## Выполнил студент группы БВТ2004 Казыханов Рамиль Ильясович

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)

### Задание №1
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [83]:
import random
import time

def gen_list(n: int, min_val: int, max_val: int) -> list[int]:
    return [random.randint(min_val, max_val) for _ in range(n)]

def join_array(arr: list) -> str:
    return '[' + ', '.join(arr) + ']'

def print_found(values, index: int):
    str_values = [str(value) for value in values]
    
    if index != -1:
        str_values[index] = f'\033[1m\033[91m{str_values[index]}\033[0m'
        
    print(join_array(str_values))
    
def value_index_pairs(data: list) -> list:
        result = []
        for i in range(len(data)):
            result.append([data[i], i])
        return result

n = int(input())
min_val = int(input())
max_val = int(input())

src_data = gen_list(n, min_val, max_val)

print(f'Исходные данные: {src_data}')

search_value = int(input())

20
0
30
Исходные данные: [30, 4, 4, 29, 4, 27, 0, 12, 2, 8, 18, 25, 30, 1, 2, 12, 11, 2, 18, 28]
11


In [84]:
#Стандартные методы
start_time = time.time()
found_index = src_data.index(search_value)
print("Время поиска: {:.6f} ms".format((time.time() - start_time) * 1000))
print_found(src_data, found_index)

Время поиска: 0.082016 ms
[30, 4, 4, 29, 4, 27, 0, 12, 2, 8, 18, 25, 30, 1, 2, 12, 11, 2, 18, 28]


In [85]:
#Бинарный поиск
def binary_search(data: list, key) -> int:
    pairs = value_index_pairs(data)
    sorted_pairs = sorted(pairs, key=lambda x: x[0])
    left = 0
    right = len(sorted_pairs) - 1
    
    while left <= right:
        mid = (left + right) >> 1
        curr_val = sorted_pairs[mid]
        
        if curr_val[0] == key:
            return curr_val[1]
        elif curr_val[0] < key:
            left = mid + 1
        else:
            right = mid - 1
    
    return -1

start_time = time.time()
found_index = binary_search(src_data, search_value)
print("Время поиска: {:.6f} ms".format((time.time() - start_time) * 1000))
print_found(src_data, found_index)

Время поиска: 0.089169 ms
[30, 4, 4, 29, 4, 27, 0, 12, 2, 8, 18, 25, 30, 1, 2, 12, 11, 2, 18, 28]


In [86]:
#Бинарное дерево
class NodeData:
    def __init__(self, value, index: int):
        self.value = value
        self.index = index

class Node:
    left = None
    right = None
    parent = None
    
    def __init__(self, data: NodeData):
        self.data = data
        
    def insert(self, item: NodeData):
        if item.value >= self.data.value:
            if self.right == None:
                self.right = Node(item)
                self.right.parent = self
            else:
                self.right.insert(item)
        else:
            if self.left == None:
                self.left = Node(item)
                self.left.parent = self
            else:
                self.left.insert(item)
            
    def index(self, item) -> int:
        if item == self.data.value:
            return self.data.index
        elif self.right != None and item > self.data.value:
            return self.right.index(item)
        elif self.left != None and item < self.data.value:
            return self.left.index(item)
        
        return -1
    
    def remove(self, item):
        if self.right != None and item > self.data.value:
            self.right.remove(item)
        elif self.left != None and item < self.data.value:
            self.left.remove(item)
        elif item == self.data.value:
            if self.left != None and self.right != None:
                if self.right.left == None:
                    self.right.parent = None
                    self.data = self.right.data
                    self.right = self.right.right
                else:
                    most_left = self.right.left
                    
                    while most_left.left != None:
                        most_left = most_left.left
                        
                    self.right.remove(most_left.data.value)
                    self.data = most_left.data
                    
            elif self.left == None and self.right == None:
                if self.parent.left == self:
                    self.parent.left = None
                else:
                    self.parent.right = None
                
                self.parent = None
                return
            
            elif self.left != None:
                self.left.parent = None
                self.data = self.left.data
                self.right = self.left.right
                self.left = self.left.left
                
            elif self.right != None:
                self.right.parent = None
                self.data = self.right.data
                self.left = self.right.left
                self.right = self.right.right

start_time = time.time()

binary_tree = Node(NodeData(src_data[0], 0))

for i in range(1, len(src_data)):
    binary_tree.insert(NodeData(src_data[i], i))
    
found_index = binary_tree.index(search_value)
print("Время поиска: {:.6f} ms".format((time.time() - start_time) * 1000))
print_found(src_data, found_index)

Время поиска: 0.156164 ms
[30, 4, 4, 29, 4, 27, 0, 12, 2, 8, 18, 25, 30, 1, 2, 12, 11, 2, 18, 28]


In [87]:
#Метод Фибоначчи
class FibonacciSearch:
    def __init__(self):
        self.__i = 0
        self.__p = 0
        self.__q = 0
        self.__is_searching = False
    
    def invoke(self, data: list, key) -> int:
        pairs = value_index_pairs(data)
        sorted_pairs = sorted(pairs, key=lambda x: x[0])
        
        self.__is_searching = True
        self.__N = len(sorted_pairs)
        self.__init_fibonacci_search()
        
        while self.__is_searching:        
            if self.__i < 0:
                self.__execute_bigger()
            elif self.__i >= self.__N:
                self.__execute_smaller()
            elif sorted_pairs[self.__i][0] < key:
                self.__execute_bigger()
            elif sorted_pairs[self.__i][0] > key:
                self.__execute_smaller()
            else:
                self.__is_searching = False
                return sorted_pairs[self.__i][1]
            
        return -1
    
    def __init_fibonacci_search(self):
        k = 0

        while self.__fibonacci(k + 1) < self.__N + 1:
            k += 1

        M = self.__fibonacci(k + 1) - (self.__N + 1)
        self.__i = self.__fibonacci(k) - M
        self.__p = self.__fibonacci(k - 1)
        self.__q = self.__fibonacci(k - 2)
        
    def __execute_bigger(self):
        if self.__p == 1:
            self.__is_searching = False
        else:
            self.__i = self.__i + self.__q
            self.__p = self.__p - self.__q
            self.__q = self.__q - self.__p
        
    def __execute_smaller(self):
        if self.__q == 0:
            self.__is_searching = False
        else:
            self.__i = self.__i - self.__q
            self.__p, self.__q = self.__q, self.__p - self.__q
    
    def __fibonacci(self, index: int) -> int:
        first = 0
        second = 1
        i = 0

        while i < index:
            first, second = second, first + second
            i += 1

        return first
    
    
start_time = time.time()
found_index = FibonacciSearch().invoke(src_data, search_value)
print("Время поиска: {:.6f} ms".format((time.time() - start_time) * 1000))
print_found(src_data, found_index)

Время поиска: 0.074863 ms
[30, 4, 4, 29, 4, 27, 0, 12, 2, 8, 18, 25, 30, 1, 2, 12, 11, 2, 18, 28]


In [88]:
#Интерполяционный
def interpolation_search(data: list, key) -> int:
    pairs = value_index_pairs(data)
    sorted_pairs = sorted(pairs, key=lambda x: x[0])
    left = 0
    right = len(data) - 1

    while left <= right:
        right_value = sorted_pairs[right][0]
        left_value = sorted_pairs[left][0]
        
        mid = left + (key - left_value) * (right - left) // (right_value - left_value)
        curr_val = sorted_pairs[mid]
        
        if curr_val[0] < key:
            left = mid + 1
        elif curr_val[0] > key:
            right = mid - 1
        else:
            return curr_val[1]

    return -1
    
start_time = time.time()
found_index = interpolation_search(src_data, search_value)
print("Время поиска: {:.6f} ms".format((time.time() - start_time) * 1000))
print_found(src_data, found_index)

Время поиска: 0.049829 ms
[30, 4, 4, 29, 4, 27, 0, 12, 2, 8, 18, 25, 30, 1, 2, 12, 11, 2, 18, 28]


### Задание №2
Написать соответствующие решения для заданных методов.

In [89]:
#Простое рехэширование
def linear_hash_insert(arr: list, value) -> bool:
    N = len(arr)
    v_hash = hash(value) % N
    
    if arr[v_hash] == None:
        arr[v_hash] = value
        return True
    
    next_v_hash = (v_hash + 1) % N
    while next_v_hash != v_hash:
        
        if arr[next_v_hash] == None:
            arr[next_v_hash] = value
            return True
        
        next_v_hash = (next_v_hash + 1) % N
        
    return False

arr = [None] * 20
print(arr)
linear_hash_insert(arr, 2)
linear_hash_insert(arr, 3)
linear_hash_insert(arr, 2)
linear_hash_insert(arr, 5)
linear_hash_insert(arr, 22)
linear_hash_insert(arr, 23)
print(arr)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, 2, 3, 2, 5, 22, 23, None, None, None, None, None, None, None, None, None, None, None, None]


In [90]:
#Рехэширование с помощью псевдослучайных чисел

def random_hash_insert(arr: list, value) -> bool:
    N = len(arr)
    v_hash = hash(value) % N
    
    if arr[v_hash] == None:
        arr[v_hash] = value
        return True
    
    R = 1
    next_v_hash = (v_hash + R) % N
    
    while next_v_hash != v_hash:
        
        if arr[next_v_hash] == None:
            arr[next_v_hash] = value
            return True
        
        R = (5 * R) % (4 * N)
        next_v_hash = (next_v_hash + (R >> 2)) % N
        
    return False

arr = [None] * 20
print(arr)
random_hash_insert(arr, 2)
random_hash_insert(arr, 3)
random_hash_insert(arr, 2)
random_hash_insert(arr, 5)
random_hash_insert(arr, 22)
random_hash_insert(arr, 23)
print(arr)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, 2, 3, 2, 5, None, None, None, None, 22, 23, None, None, None, None, None, None, None, None]


In [91]:
#Метод цепочек

class PrimitiveHashTable:
    class Entry:
        def __init__(self, key, value):
            self.key = key
            self.value = value
            
        def __str__(self):
            return str(self.key) + '=' + str(self.value)
    
    def __init__(self, size):
        self.size = size
        self.__buckets = [None] * size
        
    def insert(self, key, item):
        i_hash = hash(key) % self.size
        
        if self.__buckets[i_hash] == None:
            self.__buckets[i_hash] = []
    
        for entry in self.__buckets[i_hash]:
            if entry.key == key:
                self.__buckets[i_hash].remove(entry)
                break
    
        self.__buckets[i_hash].append(self.Entry(key, item))
        
    def remove(self, key):
        i_hash = hash(key) % self.size
        
        if self.__buckets[i_hash] != None:
            for entry in self.__buckets[i_hash]:
                if (entry.key == key):
                    self.__buckets[i_hash].remove(entry)
                    break
                    
    def get(self, key):
        i_hash = hash(key) % self.size
        
        if self.__buckets[i_hash] != None:
            for entry in self.__buckets[i_hash]:
                if (entry.key == key):
                    return entry.value
                
        return None
        
    def __str__(self):
        not_none_buckets = [bucket for bucket in self.__buckets if bucket is not None]
        str_buckets = [join_array(str(entry) for entry in bucket) for bucket in not_none_buckets]
        return join_array(str_buckets)
    
hash_table = PrimitiveHashTable(20)

hash_table.insert(0, 2)
hash_table.insert(0, 3)
hash_table.insert(1, 2)
hash_table.insert(2, 5)
hash_table.insert(20, 22)
hash_table.insert(21, 23)
print(f'Заполненная хеш-таблица: {hash_table}')

print(f'Получение значения из таблицы: {hash_table.get(20)}')

hash_table.remove(21)
print(f'Хеш-таблица после удаления элемента: {hash_table}')

Заполненная хеш-таблица: [[0=3, 20=22], [1=2, 21=23], [2=5]]
Получение значения из таблицы: 22
Хеш-таблица после удаления элемента: [[0=3, 20=22], [1=2], [2=5]]


### Задание №3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям
Написать программу,  которая находит хотя бы один способ решения задач.


In [20]:
def print_matrix(matrix: list[list[int]]):
    print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in matrix]))

def can_be_placed(desk: list[list[int]], x: int, y: int) -> bool:
    for i in range(len(desk)):
        if desk[y][i] == 1 or desk[i][x] == 1:
            return False
            
    c = y - x
    
    yd = [j for j in range(max(0, c), min(8, 8 + c))]
    xd = [i for i in range(max(0, -c), min(8, 8 - c))]

    for i in range(len(yd)):
        if desk[yd[i]][xd[i]] == 1:
            return False
        
    c = y - 7 + x
        
    yd = [j for j in range(min(7, 7 + c), max(-1, c - 1), -1)]
    xd = [i for i in range(max(0, c), min(8, 8 + c))]
    
    for i in range(len(yd)):
        if desk[yd[i]][xd[i]] == 1:
            return False
            
    return True

def process_queens(desk: list[list[int]], row_index: int) -> bool:
    queen_count = 0
    for row in desk:
        for cell in row:
            if cell == 1:
                queen_count += 1
                
    if queen_count == 8:
        print_matrix(desk)
        return True
    
    for i in range(len(desk[row_index])):
        if can_be_placed(desk, i, row_index):
            desk[row_index][i] = 1
            if process_queens(desk, row_index + 1):
                return True
            else:
                desk[row_index][i] = 0
    
    return False

desk = [[0 for _ in range(8)] for _ in range(8)]


process_queens(desk, 0)

   1   0   0   0   0   0   0   0
   0   0   0   0   1   0   0   0
   0   0   0   0   0   0   0   1
   0   0   0   0   0   1   0   0
   0   0   1   0   0   0   0   0
   0   0   0   0   0   0   1   0
   0   1   0   0   0   0   0   0
   0   0   0   1   0   0   0   0


True